# Feature Engineering from GeoTIFF

This notebook generates new derived geophysical features from selected aligned GeoTIFF layers. These features are designed to help the machine learning model better detect porphyry copper deposit patterns by emphasizing geological structures, boundaries, and local variations.

We focus on a small number of key layers and apply commonly used geophysical transformations such as:

- 1VD (First Vertical Derivative): Highlights edges and shallow structures in magnetic data.

- THD (Total Horizontal Derivative): Enhances magnetic boundaries, less sensitive to magnetic direction.

- Local Standard Deviation: Measures local variation or “texture”, often linked to structural complexity or alteration zones.

All outputs are saved as new .tif files under the /data/derived/ folder.

Only a few representative layers are selected (e.g., mag_uc_2_4km, gravity_iso_residual) to keep the process efficient and meaningful, following expert recommendations.

In [2]:
import os
import numpy as np
import rasterio
from rasterio import Affine
from rasterio.enums import Resampling
from scipy.ndimage import sobel, generic_filter
import matplotlib.pyplot as plt
import cv2

In [3]:
# === 1. Configuration ===
ALIGNED_DIR = "../../data/aligned"
DERIVED_DIR = "../../data/derived"
os.makedirs(DERIVED_DIR, exist_ok=True)

# Target input layers for feature engineering
key_layers = {
    "mag_uc_2_4km": True,  # Generate 1VD, THD, stddev
    "gravity_iso_residual": False,  # Generate stddev only
    "gravity_cscba": False  # Generate stddev (already has 1VD externally)
}

In [4]:
# === 2. Helper: Load raster ===
def load_raster(path):
    with rasterio.open(path) as src:
        arr = src.read(1)
        meta = src.meta.copy()
    return arr, meta

In [5]:
# === 3. 1VD (vertical derivative) ===
def compute_1vd(data, pixel_size):
    # Simple vertical approximation using sobel in Y-direction
    return sobel(data, axis=0) / pixel_size

In [6]:
# === 4. Total Horizontal Derivative (gradient magnitude) ===
def compute_thd(data, pixel_size):
    gx = sobel(data, axis=1) / pixel_size
    gy = sobel(data, axis=0) / pixel_size
    return np.hypot(gx, gy)

In [7]:
# === 5. Local standard deviation ===
def fast_local_std(img: np.ndarray, window_size: int = 3) -> np.ndarray:
    """Efficient local std using OpenCV box filter method."""
    # Ensure image is float32
    img = img.astype(np.float32)

    # Compute mean and mean of squares
    mean = cv2.boxFilter(img, ddepth=-1, ksize=(window_size, window_size), normalize=True, borderType=cv2.BORDER_REFLECT)
    sq_mean = cv2.boxFilter(img**2, ddepth=-1, ksize=(window_size, window_size), normalize=True, borderType=cv2.BORDER_REFLECT)

    # Standard deviation
    std = np.sqrt(sq_mean - mean**2)
    return std


In [8]:
# === 6. Batch feature generation ===
for layer, generate_all in key_layers.items():
    aligned_path = os.path.join(ALIGNED_DIR, f"{layer}_aligned.tif")
    data, meta = load_raster(aligned_path)
    transform = meta['transform']
    pixel_size = abs(transform[0])  # assume square pixel

    if generate_all:
        # 1VD
        vd = compute_1vd(data, pixel_size)
        meta.update(dtype='float32', compress='lzw')
        with rasterio.open(os.path.join(DERIVED_DIR, f"{layer}_1vd.tif"), 'w', **meta) as dst:
            dst.write(vd.astype(np.float32), 1)

        # THD
        thd = compute_thd(data, pixel_size)
        with rasterio.open(os.path.join(DERIVED_DIR, f"{layer}_thd.tif"), 'w', **meta) as dst:
            dst.write(thd.astype(np.float32), 1)

    # Local stddev (faster method)
    stddev = fast_local_std(data, window_size=3)
    with rasterio.open(os.path.join(DERIVED_DIR, f"{layer}_stddev3x3.tif"), 'w', **meta) as dst:
        dst.write(stddev.astype(np.float32), 1)

print("Key feature engineering GeoTIFFs generated in", DERIVED_DIR)

C:\Users\John\AppData\Local\Temp\ipykernel_21848\3953158862.py:12: RuntimeWarning: invalid value encountered in sqrt
  std = np.sqrt(sq_mean - mean**2)


Key feature engineering GeoTIFFs generated in ../../data/derived
